In [1]:
import pandas as pd

In [13]:
def processar_dados_ipeadata(url: str) -> pd.DataFrame:
    try:
        tabelas = pd.read_html(url)
        dados = tabelas[2]

        dados = dados.drop(0)

        dados.columns = ['Data', 'Preço']

        # Converte a coluna 'Data' para o tipo datetime
        dados['Data'] = pd.to_datetime(dados['Data'], format='%d/%m/%Y')

        # Converte a coluna 'Preço' para float, tratando os separadores de milhar e decimal
        dados['Preço'] = dados['Preço'].str.replace('.', '', regex=False).str.replace(',', '.', regex=False).astype(float)

        # Converte os valores de 'Preço' para formato percentual e ajusta para a vírgula como separador decimal
        dados['Preço'] = (dados['Preço'] / 100).apply(lambda x: f"{x:.2f}".replace('.', ','))

        # Ordena a coluna 'Data' de forma ascendente
        dados = dados.sort_values(by='Data').reset_index(drop=True)

        return dados

    except Exception as e:
        print(f"Erro ao processar os dados: {e}")
        return pd.DataFrame()

url = "http://www.ipeadata.gov.br/ExibeSerie.aspx?module=m&serid=1650971490&oper=view"

df = processar_dados_ipeadata(url)
print(df.head())


        Data  Preço
0 1987-05-20  18,63
1 1987-05-21  18,45
2 1987-05-22  18,55
3 1987-05-25  18,60
4 1987-05-26  18,63


In [14]:
print(df.tail())

            Data  Preço
11287 2024-10-29  71,09
11288 2024-10-30  73,21
11289 2024-10-31  73,25
11290 2024-11-01  73,63
11291 2024-11-04  74,89


In [15]:
print(df.shape)

(11292, 2)


In [16]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11292 entries, 0 to 11291
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Data    11292 non-null  datetime64[ns]
 1   Preço   11292 non-null  object        
dtypes: datetime64[ns](1), object(1)
memory usage: 176.6+ KB
None


In [17]:
print(df.describe())

                                Data
count                          11292
mean   2006-10-17 20:03:11.285865984
min              1987-05-20 00:00:00
25%              1998-07-01 18:00:00
50%              2007-05-27 12:00:00
75%              2015-03-23 06:00:00
max              2024-11-04 00:00:00


In [18]:
print(df.isnull().sum())

Data     0
Preço    0
dtype: int64


In [19]:
print(df['Preço'].value_counts())

Preço
18,48    26
18,15    22
16,85    20
18,63    18
17,00    18
         ..
49,51     1
47,72     1
37,04     1
40,02     1
73,25     1
Name: count, Length: 5303, dtype: int64


In [21]:
print(df.duplicated().sum())

0


In [23]:
df.to_csv('dados_ipeadata.csv', index=False, sep=';', encoding='utf-8')